In [1]:
import os
import pandas as pd

os.sys.path.insert(0, '../evaluation')

from evaluate import evaluate_system

# BLEU

https://github.com/moses-smt/mosesdecoder/blob/master/scripts/generic/multi-bleu.perl

Download multi-bleu.perl and put it in **../evaluation/tools/**

# METEOR

http://www.cs.cmu.edu/~alavie/METEOR/

Download meteor-1.5 and unpack into **../evaluation/tools/**

The jar path must be **../evaluation/tools/meteor-1.5/meteor-1.5.jar**

# TER

http://www.cs.umd.edu/~snover/tercom/

Download tercom-0.7.25 and unpack into **../evaluation/tools/**

The jar path must be **../evaluation/tools/tercom-0.7.25/tercom-0.7.25.jar**

<img src="../img/competition_results.jpg" />

## Melbourne

<a href="../data/models/melbourne/WebNLG Challenge - Bayu Distiawan T.pdf">report</a>

In [3]:
out_filepath = evaluate_system('melbourne')

df = pd.read_csv(out_filepath)
df

,subset,references,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
0,1size,"[0, 1, 2]",63.81,87.5,72.5,60.5,49.9,0.471922,0.274120
1,2size,"[0, 1, 2]",51.25,82.2,59.7,44.8,34.0,0.421393,0.389833
2,3size,"[0, 1, 2]",43.42,80.3,54.7,37.4,25.6,0.376657,0.470233
3,4size,"[0, 1, 2]",42.14,78.6,52.7,35.6,24.6,0.358887,0.520130
4,5size,"[0, 1, 2]",36.72,75.3,47.9,29.7,18.4,0.327765,0.579703
5,6size,"[0, 1, 2]",52.16,81.8,60.3,45.7,35.3,0.378531,0.469173
6,7size,"[0, 1, 2]",48.65,81.4,60.3,44.7,33.6,0.355614,0.467217
7,all-cat,"[0, 1, 2]",45.13,79.9,55.4,38.8,27.4,0.376396,0.474403
8,new-cat,"[0, 1, 2]",33.27,72.2,44.0,27.1,16.2,0.332213,0.559212
9,old-cat,"[0, 1, 2]",54.52,86.3,65.0,48.5,36.6,0.414642,0.402598


In [4]:
!head -100 ../data/models/melbourne/melbourne.txt | tail -10

## Tilb-SMT

<a href="../data/models/tilburg-smt/report.pdf">report</a>

In [6]:
out_filepath = evaluate_system('tilburg-smt')

df = pd.read_csv(out_filepath)
df

,subset,references,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
0,1size,"[0, 1, 2]",52.16,88.4,70.9,56.0,45.7,0.412354,0.367070
1,2size,"[0, 1, 2]",48.16,76.7,57.7,43.6,33.6,0.399773,0.464433
2,3size,"[0, 1, 2]",43.42,73.2,51.9,36.8,26.2,0.380756,0.535855
3,4size,"[0, 1, 2]",42.44,72.2,50.5,35.3,25.2,0.372728,0.573584
4,5size,"[0, 1, 2]",38.68,71.2,47.8,31.4,20.9,0.361045,0.615279
5,6size,"[0, 1, 2]",49.71,78.6,59.2,42.8,30.7,0.404062,0.565714
6,7size,"[0, 1, 2]",44.44,75.8,53.5,37.0,26.0,0.390495,0.636231
7,all-cat,"[0, 1, 2]",44.28,74.4,53.0,37.5,26.8,0.380463,0.537692
8,new-cat,"[0, 1, 2]",29.88,65.0,40.6,24.5,14.9,0.333329,0.617078
9,old-cat,"[0, 1, 2]",54.29,81.8,62.6,47.4,35.8,0.421025,0.470480


In [8]:
!head -100 ../data/models/tilburg-smt/tilburg-smt.txt | tail -10

novel castle is english language 
. eric flint was born in burbank , california . 
macmillan publishers is the parent company of farrar , straus and giroux 
john cowper powys notablework a glastonbury romance 
soho press , united states . 
, published by faber and faber 
asian americans are an ethnic group . 
english is the language of the united states . 
weymouth sands was preceded by a glastonbury romance 
the manager of a.c. chievo verona is rolando maran 


## PKUWriter

<a href="../data/models/pkuwriter/PKUWriter_system.pdf">report</a>

In [7]:
out_filepath = evaluate_system('pkuwriter')

df = pd.read_csv(out_filepath)
df

,subset,references,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
0,1size,"[0, 1, 2]",53.15,77.3,59.1,46.8,37.4,0.415371,0.371388
1,2size,"[0, 1, 2]",45.34,75.4,53.6,40.9,31.7,0.355510,0.481268
2,3size,"[0, 1, 2]",34.89,70.7,44.5,29.8,20.3,0.301129,0.602554
3,4size,"[0, 1, 2]",36.80,73.1,48.6,34.3,24.9,0.292479,0.589683
4,5size,"[0, 1, 2]",34.17,72.4,45.9,30.7,21.3,0.274453,0.623402
5,6size,"[0, 1, 2]",52.89,84.1,64.5,49.1,37.6,0.389259,0.425865
6,7size,"[0, 1, 2]",52.63,84.8,62.8,45.0,34.1,0.380657,0.469160
7,all-cat,"[0, 1, 2]",39.85,73.9,49.9,35.6,26.0,0.317300,0.553278
8,new-cat,"[0, 1, 2]",25.36,63.3,35.7,22.0,13.5,0.248942,0.674590
9,old-cat,"[0, 1, 2]",51.18,82.0,60.7,45.9,35.4,0.377063,0.450567


In [9]:
!head -100 ../data/models/pkuwriter/pkuwriter.txt | tail -10

the novel into battle is written by the english language .
john cowper powys was born in burbank , california .
the parent company of the macmillan publishers press is the macmillan publishers company of the macmillan publishers .
the novel owen glendower is a notable work by john cowper powys .
soho press is located in united states .
the polish academy for writtenwork is referencenumber _ in _ the _ national _ register _ of _ historic _ places .
asian americans are an ethnic group in the united states .
english is the language spoken in the united states .
the novel into battle is preceded by a glastonbury romance .
rolando maran manages the a . c . chievo verona .


## UPF-FORGe

<a href="../data/models/upf-forge/WebNLG_V0.2.pdf">report</a>

In [10]:
out_filepath = evaluate_system('upf-forge')

df = pd.read_csv(out_filepath)
df

,subset,references,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
0,1size,"[0, 1, 2]",60.29,86.5,68.7,55.3,44.8,0.469279,0.270624
1,2size,"[0, 1, 2]",43.24,77.5,52.6,35.9,23.9,0.409700,0.467734
2,3size,"[0, 1, 2]",36.84,74.8,46.9,29.5,17.8,0.383582,0.567642
3,4size,"[0, 1, 2]",35.54,73.6,45.8,28.3,16.8,0.376883,0.622324
4,5size,"[0, 1, 2]",32.66,72.4,44.0,25.1,14.2,0.367748,0.665456
5,6size,"[0, 1, 2]",38.33,76.5,49.5,30.6,18.6,0.402804,0.598195
6,7size,"[0, 1, 2]",37.84,75.7,48.8,30.1,18.4,0.399411,0.654687
7,all-cat,"[0, 1, 2]",38.65,75.5,48.7,31.1,19.5,0.391077,0.559327
8,new-cat,"[0, 1, 2]",35.70,74.2,45.7,28.3,16.9,0.373621,0.559565
9,old-cat,"[0, 1, 2]",40.88,76.4,51.1,33.3,21.5,0.405867,0.559126


In [11]:
!head -100 ../data/models/upf-forge/upf-forge.txt | tail -10

the english language is spoken in castle ( novel ) .
eric flint was born in burbank ( california ) .
the parent company of farrar , straus and giroux is macmillan publishers .
a glastonbury romance wrote john cowper powys .
soho press is published in the united states .
the publisher of the secret scripture is faber and faber .
a ethnic group of the united states are asian americans .
the english language is spoken in the united states .
weymouth sands follows a glastonbury romance .
rolando maran is the manager of a.c. chievo verona .


## Tilb-Pipeline

<a href="../data/models/tilburg-template/report.pdf">report</a>

In [12]:
out_filepath = evaluate_system('tilburg-template')

df = pd.read_csv(out_filepath)
df

,subset,references,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
0,1size,"[0, 1, 2]",44.81,90.8,72.5,56.6,43.6,0.323192,0.475854
1,2size,"[0, 1, 2]",34.84,85.2,61.8,44.5,32.4,0.318193,0.515267
2,3size,"[0, 1, 2]",33.91,82.0,56.3,38.3,26.3,0.307431,0.559991
3,4size,"[0, 1, 2]",34.05,81.3,55.8,38.6,26.7,0.297620,0.578120
4,5size,"[0, 1, 2]",33.03,78.2,51.0,33.0,21.1,0.296692,0.629880
5,6size,"[0, 1, 2]",39.83,77.8,51.3,33.5,21.1,0.358336,0.571128
6,7size,"[0, 1, 2]",40.68,77.3,52.0,32.9,20.7,0.387253,0.604177
7,all-cat,"[0, 1, 2]",35.29,81.8,56.6,38.8,26.5,0.308340,0.566023
8,new-cat,"[0, 1, 2]",20.65,81.8,57.1,39.3,26.6,0.216254,0.657518
9,old-cat,"[0, 1, 2]",44.34,81.8,56.4,38.6,26.5,0.382392,0.488559


In [13]:
!head -100 ../data/models/tilburg-template/tilburg-template.txt | tail -10

castle is in english .
eric flint was born in burbank , california .
macmillan publishers owns farrar , straus , and giroux .
a glastonbury romance is a notable work by john cowper powys .
soho press is in united states .
the secret scripture publisher of faber and faber .
asian americans live in united states .
united states is in english .
weymouth sands was preceded by a glastonbury romance .
rolando maran manages a . c . chievo verona .


## Tilb-NMT

<a href="../data/models/tilburg-nmt/report.pdf">report</a>

In [13]:
out_filepath = evaluate_system('tilburg-nmt')

df = pd.read_csv(out_filepath)
df

,subset,references,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
0,1size,"[0, 1, 2]",50.33,74.2,56.6,44.2,34.5,0.447765,0.429996
1,2size,"[0, 1, 2]",37.13,67.5,44.4,30.5,20.8,0.384220,0.584420
2,3size,"[0, 1, 2]",31.85,66.4,39.9,25.0,15.5,0.345341,0.646517
3,4size,"[0, 1, 2]",32.92,68.9,41.5,25.8,16.0,0.330702,0.622947
4,5size,"[0, 1, 2]",28.64,67.9,37.9,21.7,12.1,0.306137,0.665970
5,6size,"[0, 1, 2]",39.88,73.8,48.8,32.6,21.9,0.346942,0.585564
6,7size,"[0, 1, 2]",34.26,75.3,44.4,26.9,16.7,0.338627,0.616804
7,all-cat,"[0, 1, 2]",34.60,68.8,42.7,27.5,17.7,0.348784,0.609501
8,new-cat,"[0, 1, 2]",25.12,59.3,32.9,19.2,10.6,0.310575,0.721628
9,old-cat,"[0, 1, 2]",43.28,77.7,52.1,35.5,24.4,0.382624,0.514568


In [14]:
!head -100 ../data/models/tilburg-nmt/tilburg-nmt.txt | tail -10

english is the language of castle . 
eric flint was born in burbank , california . 
macmillan publishers is the parent company of farrar , straus , and giroux . 
a glastonbury romance is a notable work by john cowper powys . 
soho press is located in united states . 
th secret scriptures was published by faber and faber . 
asian americans are an ethnic group in united states . 
english is the language of united states . 
weymouth sands was preceded by a glastonbury romance . 
the manager of a.c. chievo verona is rolando maran . 


## Baseline

In [15]:
out_filepath = evaluate_system('baseline_sorted')

df = pd.read_csv(out_filepath)
df

,subset,references,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
0,1size,"[0, 1, 2]",42.36,65.1,45.2,38.8,33.6,0.277388,0.491072
1,2size,"[0, 1, 2]",33.83,65.2,42.1,33.2,26.9,0.241359,0.575508
2,3size,"[0, 1, 2]",30.46,64.3,40.1,29.5,22.2,0.222312,0.639728
3,4size,"[0, 1, 2]",31.11,64.1,40.8,30.1,23.2,0.221627,0.653365
4,5size,"[0, 1, 2]",28.88,63.2,38.9,26.7,19.8,0.210370,0.669054
5,6size,"[0, 1, 2]",53.25,85.7,67.1,51.9,41.1,0.368557,0.426767
6,7size,"[0, 1, 2]",53.81,89.9,69.3,54.3,43.1,0.378776,0.402137
7,all-cat,"[0, 1, 2]",33.24,65.4,42.3,31.8,24.8,0.235436,0.613080
8,new-cat,"[0, 1, 2]",6.13,38.9,10.1,4.9,2.8,0.075353,0.807292
9,old-cat,"[0, 1, 2]",52.39,81.2,61.1,47.1,36.8,0.377720,0.448648


In [16]:
!head -100 ../data/models/baseline_sorted/baseline_sorted.txt | tail -10

the english language is spoken in the republic of ireland .
author was born in burbank , california .
the parent company of wiley - blackwell is macmillan publishers .
a glastonbury romance was written by author .
lahore is in the united states .
the parent company of wiley - blackwell is parentcompany .
asian americans are an ethnic group in the united states .
english is the language of the united states .
a glastonbury romance was preceded by precededby .
the manager of a . c . chievo verona is rolando maran .


## Adapt

<a href="../data/models/adaptcentre/WebNLG_system.pdf">report</a>

In [17]:
out_filepath = evaluate_system('adaptcentre')

df = pd.read_csv(out_filepath)
df

,subset,references,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
0,1size,"[0, 1, 2]",48.13,69.6,51.3,42.2,35.5,0.378119,0.442952
1,2size,"[0, 1, 2]",33.77,58.3,37.8,27.9,21.2,0.326372,0.727348
2,3size,"[0, 1, 2]",27.78,52.1,32.5,22.2,15.8,0.306462,0.935833
3,4size,"[0, 1, 2]",27.56,50.3,32.0,22.2,16.1,0.296947,0.982271
4,5size,"[0, 1, 2]",26.35,50.8,31.4,20.8,14.6,0.290825,0.996950
5,6size,"[0, 1, 2]",62.03,86.5,69.8,56.4,45.6,0.432208,0.370827
6,7size,"[0, 1, 2]",62.96,88.7,71.5,56.0,44.4,0.435427,0.372997
7,all-cat,"[0, 1, 2]",31.06,54.9,35.7,25.3,18.7,0.315980,0.849266
8,new-cat,"[0, 1, 2]",10.53,34.5,14.7,7.1,3.4,0.192333,1.404981
9,old-cat,"[0, 1, 2]",60.59,85.6,67.9,53.7,43.1,0.445389,0.378763


In [18]:
!head -100 ../data/models/adaptcentre/adaptcentre.txt | tail -10

the novel castle is the language spoken in the english language .
eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus and giroux .
a glastonbury romance is the notable work of john cowper powys .
soho press is located in the united states .
the novel the secret scriptures , is published by faber and faber .
asian americans are an ethnic group in the u.s .
english is the language of the united states .
a glastonbury romance is the sequel to weymouth sands .
the manager of a.c . chievo verona is rolando maran .


## UIT-VNU

<a href="../data/models/uit-vietnam/Phong-Nguyen-Tran_Dang-Tuan-Nguyen.docx">report</a>

In [19]:
out_filepath = evaluate_system('uit-vietnam')

df = pd.read_csv(out_filepath)
df

,subset,references,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
0,1size,"[0, 1, 2]",44.65,70.7,52.4,38.1,28.2,0.328469,0.625150
1,2size,"[0, 1, 2]",16.52,68.6,53.4,38.8,29.1,0.156637,0.764153
2,3size,"[0, 1, 2]",5.69,69.2,56.9,40.4,29.3,0.091948,0.831528
3,4size,"[0, 1, 2]",0.31,61.3,57.9,43.1,32.9,0.042366,0.866558
4,5size,"[0, 1, 2]",0.01,54.5,59.9,42.2,29.3,0.021128,0.896699
5,6size,"[0, 1, 2]",0.00,0.0,0.0,0.0,0.0,0.000000,0.929323
6,7size,"[0, 1, 2]",0.00,0.0,0.0,0.0,0.0,0.000000,0.929577
7,all-cat,"[0, 1, 2]",7.07,67.5,54.4,39.5,29.2,0.099207,0.827492
8,new-cat,"[0, 1, 2]",0.11,49.6,53.0,37.6,26.3,0.033486,0.877597
9,old-cat,"[0, 1, 2]",19.87,72.0,54.7,39.8,29.7,0.153499,0.785071


In [20]:
!head -100 ../data/models/uit-vietnam/uit-vietnam.txt | tail -10

# All results

In [36]:
import glob

models = [os.path.basename(p) for p in glob.glob('../data/models/*')]

dfs = [pd.read_csv(f'../data/models/{model}/system_evaluation.csv', index_col=['subset', 'references']) for model in models]

df = pd.concat(dfs, keys=models)

df.head()

bleu  bleu_1  bleu_2  bleu_3  bleu_4    meteor  \
      subset references                                                    
abe-1 1size  [0, 1, 2]   63.31    91.5    74.5    60.6    50.7  0.484139   
      2size  [0, 1, 2]   47.49    75.7    55.8    40.8    29.5  0.421946   
      3size  [0, 1, 2]   40.64    70.5    49.1    34.0    23.1  0.392538   
      4size  [0, 1, 2]   39.95    68.4    48.0    33.5    23.2  0.388002   
      5size  [0, 1, 2]   36.81    67.1    45.6    30.3    19.8  0.374754   

                              ter  
      subset references            
abe-1 1size  [0, 1, 2]   0.245742  
      2size  [0, 1, 2]   0.457006  
      3size  [0, 1, 2]   0.581003  
      4size  [0, 1, 2]   0.626860  
      5size  [0, 1, 2]   0.681290

In [37]:
df.loc[(slice(None), 'all-cat', '[0, 1, 2]'), :].sort_values('bleu', ascending=False)

,,,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
,subset,references,,,,,,,
melbourne,all-cat,"[0, 1, 2]",45.13,79.9,55.4,38.8,27.4,0.376396,0.474403
tilburg-smt,all-cat,"[0, 1, 2]",44.28,74.4,53.0,37.5,26.8,0.380463,0.537692
abe-1,all-cat,"[0, 1, 2]",42.33,71.1,50.7,35.7,25.0,0.400918,0.573666
pkuwriter,all-cat,"[0, 1, 2]",39.85,73.9,49.9,35.6,26.0,0.317300,0.553278
upf-forge,all-cat,"[0, 1, 2]",38.65,75.5,48.7,31.1,19.5,0.391077,0.559327
tilburg-template,all-cat,"[0, 1, 2]",35.29,81.8,56.6,38.8,26.5,0.308340,0.566023
tilburg-nmt,all-cat,"[0, 1, 2]",34.60,68.8,42.7,27.5,17.7,0.348784,0.609501
baseline_sorted,all-cat,"[0, 1, 2]",33.24,65.4,42.3,31.8,24.8,0.235436,0.613080
adaptcentre,all-cat,"[0, 1, 2]",31.06,54.9,35.7,25.3,18.7,0.315980,0.849266


In [38]:
df.loc[(slice(None), 'old-cat', '[0, 1, 2]'), :].sort_values('bleu', ascending=False)

,,,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
,subset,references,,,,,,,
adaptcentre,old-cat,"[0, 1, 2]",60.59,85.6,67.9,53.7,43.1,0.445389,0.378763
melbourne,old-cat,"[0, 1, 2]",54.52,86.3,65.0,48.5,36.6,0.414642,0.402598
tilburg-smt,old-cat,"[0, 1, 2]",54.29,81.8,62.6,47.4,35.8,0.421025,0.470480
baseline_sorted,old-cat,"[0, 1, 2]",52.39,81.2,61.1,47.1,36.8,0.377720,0.448648
pkuwriter,old-cat,"[0, 1, 2]",51.18,82.0,60.7,45.9,35.4,0.377063,0.450567
abe-1,old-cat,"[0, 1, 2]",48.37,75.3,56.1,41.7,31.0,0.425713,0.531070
tilburg-template,old-cat,"[0, 1, 2]",44.34,81.8,56.4,38.6,26.5,0.382392,0.488559
tilburg-nmt,old-cat,"[0, 1, 2]",43.28,77.7,52.1,35.5,24.4,0.382624,0.514568
upf-forge,old-cat,"[0, 1, 2]",40.88,76.4,51.1,33.3,21.5,0.405867,0.559126


In [39]:
df.loc[(slice(None), 'new-cat', '[0, 1, 2]'), :].sort_values('bleu', ascending=False)

,,,bleu,bleu_1,bleu_2,bleu_3,bleu_4,meteor,ter
,subset,references,,,,,,,
upf-forge,new-cat,"[0, 1, 2]",35.70,74.2,45.7,28.3,16.9,0.373621,0.559565
abe-1,new-cat,"[0, 1, 2]",34.40,65.9,43.8,28.1,17.3,0.372320,0.623977
melbourne,new-cat,"[0, 1, 2]",33.27,72.2,44.0,27.1,16.2,0.332213,0.559212
tilburg-smt,new-cat,"[0, 1, 2]",29.88,65.0,40.6,24.5,14.9,0.333329,0.617078
pkuwriter,new-cat,"[0, 1, 2]",25.36,63.3,35.7,22.0,13.5,0.248942,0.674590
tilburg-nmt,new-cat,"[0, 1, 2]",25.12,59.3,32.9,19.2,10.6,0.310575,0.721628
tilburg-template,new-cat,"[0, 1, 2]",20.65,81.8,57.1,39.3,26.6,0.216254,0.657518
adaptcentre,new-cat,"[0, 1, 2]",10.53,34.5,14.7,7.1,3.4,0.192333,1.404981
baseline_sorted,new-cat,"[0, 1, 2]",6.13,38.9,10.1,4.9,2.8,0.075353,0.807292
